# Simulating a single cell

This notebook shows how to simulate a single cell using the `Cell` class.

## Compiling mechanisms

Before loading the cell, the cell mechanisms need to be compiled and provided. This compilation step is a performance requirement.

Refer to the [MOD2IR: High-Performance Code Generation for a Biophysically Detailed Neuronal Simulation DSL](https://dl.acm.org/doi/abs/10.1145/3578360.3580268) publication to learn more about the compilation process.


If the mechanisms are not compiled, you need to run the `nrnivmodl` command to perform the compilation.

The mechanisms are already available at the examples directory.

With NEURON installed, simply run the following command for the compilation.


In [1]:
!nrnivmodl ../mechanisms

/home/tuncel/git-repos/BlueCelluLab/examples/1-singlecell
Mod files: "../mechanisms/../mechanisms/CaDynamics_DC0.mod" "../mechanisms/../mechanisms/CaDynamics_E2.mod" "../mechanisms/../mechanisms/Ca_HVA2.mod" "../mechanisms/../mechanisms/Ca_HVA.mod" "../mechanisms/../mechanisms/Ca_LVAst.mod" "../mechanisms/../mechanisms/Ca.mod" "../mechanisms/../mechanisms/DetAMPANMDA.mod" "../mechanisms/../mechanisms/DetGABAAB.mod" "../mechanisms/../mechanisms/gap.mod" "../mechanisms/../mechanisms/GluSynapse.mod" "../mechanisms/../mechanisms/Ih.mod" "../mechanisms/../mechanisms/Im.mod" "../mechanisms/../mechanisms/KdShu2007.mod" "../mechanisms/../mechanisms/K_Pst.mod" "../mechanisms/../mechanisms/K_Tst.mod" "../mechanisms/../mechanisms/Nap_Et2.mod" "../mechanisms/../mechanisms/NaTa_t.mod" "../mechanisms/../mechanisms/NaTg.mod" "../mechanisms/../mechanisms/NaTs2_t.mod" "../mechanisms/../mechanisms/netstim_inhpoisson.mod" "../mechanisms/../mechanisms/ProbAMPANMDA_EMS.mod" "../mechanisms/../mechanisms/Pro

**Note**: The compiled mechanisms need to be provided before importing bluecellulab.

In [2]:
from pathlib import Path

from matplotlib import pyplot as plt
from matplotlib.pyplot import get_cmap

from bluecellulab import Cell, Simulation
from bluecellulab.circuit.circuit_access import EmodelProperties

from bluecellulab.stimulus.circuit_stimulus_definitions import OrnsteinUhlenbeck, ShotNoise
from bluecellulab import RNGSettings

## Loading the cell

A Cell is consisting of the two following components

- Electrophysiology - represented as hoc
- Morphology (shape) - represented as asc in this tutorial

In [ ]:
hoc_file = Path("hoc") / "cADpyr_L2TPC.hoc"
morph_file = Path("morphology") / "rr110330_C3_idA.asc"

This command will compile the mechanisms files. The compilation step

In [ ]:
morph_file

In [ ]:
emodel_properties = EmodelProperties(threshold_current=1.1433533430099487,
                                     holding_current=1.4146618843078613,
                                     AIS_scaler=1.4561502933502197)
cell = Cell(hoc_file, morph_file, template_format="v6", emodel_properties=emodel_properties)

In [ ]:
cell

### Exploring cell's attributes

Morphology related attributes such as the Radius (Ra) of a section can be accessed as follows:

In [ ]:
cell.soma.Ra

To see the children of a section, you can use the `children` attribute.

In [ ]:
cell.soma.children()

Properties of other sections such as length(L) can also be displayed as follows:

In [ ]:
cell.apical[0].Ra

In [ ]:
cell.apical[0].L

Electrophysiological attributes such as the holding and threshold currents of the cell can be accessed as follows:

In [ ]:
cell.hypamp  # holding current

In [ ]:
cell.threshold

## Simulation

In this step we will apply some stimulation to the cell and run a simulation.

Here we create a Simulation object and add the cell we wish to simulate to it.

In [ ]:
sim = Simulation()
sim.add_cell(cell)

### Adding a step current

Here a step current is set to be given to the cell during the simulation.

In [ ]:
cell.add_step(start_time=15.0, stop_time=25.0, level=1.0)

To run the cell, we call the `run` method of the simulation object.

In [ ]:
sim.run(45, cvode=False)

Retrieving the time and voltage values of the simulation is done as follows:

In [ ]:
time, voltage = cell.get_time(), cell.get_soma_voltage()

**Note**: To retrieve the voltage we used `cell.get_soma_voltage()`. In the next steps we will see how to retrieve the voltage from other sections.

In [ ]:
plt.plot(time, voltage)
plt.xlabel("Time (ms)")
plt.ylabel("Voltage (mV)")

Here at the beginning we see a single spike and then we can clearly observe the impact of the step current we injected between 15 and 25 ms.

### Adding a more complex stimulation

Besides the step current, we can also add more complex stimulation to the cell.
In this example we will add a Poisson shot noise current to the cell.

Here we create a new Cell instance since the underlying NEURON implementation leaves some state after adding stimulus to the cell.

We do not want the previously injected step current to interfere with the new stimulus.

In [ ]:
cell = Cell(hoc_file, morph_file, template_format="v6", emodel_properties=emodel_properties)
sim = Simulation()
sim.add_cell(cell)

Here to define the ShotNoise stimulus, we need to create an object of type ShotNoise. The attributes of the stimulus object (ShotNoise object) are checked via a dynamic type checker to early detect errors and misuses.

In [ ]:
shotnoise_stimulus = ShotNoise(
    target="single-cell", delay=15, duration=20,
    rise_time=0.4, decay_time=4, rate=2E3, amp_mean=40E-3, amp_var=16E-4,
    seed=3899663
)

Since ShotNoise is a stochastic signal, we need to set the rng_settings attribute of the cell.

In [ ]:
rng_obj = RNGSettings.get_instance().set_seeds(base_seed=42)

Here we add the shotnoise to the middle point (denoted by 0.5) of soma 

In [ ]:
time_vec, stim_vec = cell.add_replay_shotnoise(
    cell.soma, 0.5,
    shotnoise_stimulus,
    shotnoise_stim_count=3)
time_vec = time_vec.to_python()
stim_vec = stim_vec.to_python()

Here is the current we injected to the cell.

In [ ]:
plt.plot(time_vec, stim_vec)
plt.xlabel("Time (ms)")
plt.ylabel("Current (nA)")

In [ ]:
sim.cells

In [ ]:
sim.cells[0] is cell

In [ ]:
sim.run(45, cvode=False)
time, voltage = cell.get_time(), cell.get_soma_voltage()

The figure below shows how the cell responded to the shotnoise. 

As in the previous stimulus, there is a single spike at the beginning and the impact of shotnoise begins at 15 ms and lasts for 20ms.

Compared to the previous voltage figure, here we can tell that the shotnoise has a different effect on the cell.

In [ ]:
plt.plot(time, voltage)
plt.xlabel("Time (ms)")
plt.ylabel("Voltage (mV)")

### Using the spike detector

The spike detector can be used to detect the exact time of the spikes.

In [ ]:
cell = Cell(hoc_file, morph_file, template_format="v6", emodel_properties=emodel_properties)
tstim = cell.add_ramp(start_time=10.0, stop_time=30.0, start_level=0.5, stop_level=4.0)
sim = Simulation()
sim.add_cell(cell)

Before running the simulation, we need to add the spike detector via `start_recording_spikes` to the cell and set the threshold voltage.

Above the threshold is considered a spike. 

`get_recorded_spikes` returns the time of the spikes from the section "soma".

In [ ]:
cell.start_recording_spikes(None, location="soma", threshold=-10)
sim.run(45, cvode=False)
spikes = cell.get_recorded_spikes(location="soma", threshold=-10)
time, voltage = cell.get_time(), cell.get_soma_voltage()

In [ ]:
print(f"spike times: {spikes}")

The detected spikes are denoted as red dashed lines in the voltage plot.

In [ ]:
plt.plot(time, voltage)
plt.xlabel("Time (ms)")
plt.ylabel("Voltage (mV)")
# add spikes to the plot as dashed lines
for spike in spikes:
    plt.axvline(spike, c="r", linestyle="--")

### Recording from another section

Besides soma, the voltage can be recorded from other sections as well.
Here we record from the axon initial segment (ais) and plot the voltage from both soma and AIS.

In [ ]:
cell = Cell(hoc_file, morph_file, template_format="v6", emodel_properties=emodel_properties)
cell.add_ramp(start_time=10.0, stop_time=30.0, start_level=0.5, stop_level=4.0)
sim = Simulation()
sim.add_cell(cell)
cell.add_ais_recording(dt=cell.record_dt)

In [ ]:
sim.run(45, cvode=False)

In [ ]:
time = cell.get_time()
soma_voltage = cell.get_soma_voltage()
ais_voltage = cell.get_ais_voltage()

The dotted line shows the recordings from the axon initial segment.

In [ ]:
plt.plot(time, soma_voltage, label="soma")
plt.plot(time, ais_voltage, label="ais", linestyle="--")
plt.legend()
plt.xlabel("Time (ms)")
plt.ylabel("Voltage (mV)")

### Recording from all sections

In this step we will go one level beyond and record from every section of the neuron.

This time we will add an Ornstein-Uhlenbeck process current to the cell.

In [ ]:
cell = Cell(hoc_file, morph_file, template_format="v6", emodel_properties=emodel_properties)
stimulus = OrnsteinUhlenbeck(
            target="single-cell", delay=15, duration=20,
            tau=2.8, sigma=0.64, mean=0.729, mode="current_clamp", dt=0.25, seed=1
        )
rng_obj = RNGSettings.get_instance().set_seeds(base_seed=42)
sim = Simulation()
sim.add_cell(cell)

With `add_allsections_voltagerecordings` we enable recording from every section of the cell.

In [ ]:
cell.add_allsections_voltagerecordings()

In [ ]:
time_vec, stim_vec = cell.add_ornstein_uhlenbeck(cell.soma, 0.5, stimulus, stim_count=1)


The Ornstein-Uhlenbeck signal is plotted below.

In [ ]:
plt.plot(time_vec, stim_vec)
plt.xlabel("Time (ms)")
plt.ylabel("Current (nA)")

In [ ]:
sim.run(45, cvode=False)

To get the recorded voltage from all sections, we use `get_allsections_voltage`.

In [ ]:
recordings = cell.get_allsections_voltagerecordings()
time = cell.get_time()

Here in order to display how the voltage changes in different sections, we plot the voltage of the soma and its subsections.

We use the `subtree` method to get the subsections.

In [ ]:
subtree = [x.name() for x in cell.soma.subtree()][:10]

In [ ]:
# Define the figure size
plt.figure(figsize=(8, 8))

# Define colormap
color_map = get_cmap('viridis', len(subtree))

# plot the voltage recordings from the subtree
for i, section in enumerate(subtree):
    voltage = recordings[section]
    plt.plot(time, voltage, label=section.split(".")[-1], color=color_map(i))
    
# display legend only once
plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1.0))
plt.xlabel("Time (ms)")
plt.ylabel("Voltage (mV)")
plt.show()


In the plot above, the colour intensity is given by the order of subtree.